# Unidad 03 - Ejercicios

In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
foreach (var line in System.IO.File.ReadLines(".env"))
{
    var parts = line.Split('=');
    if (parts.Length == 2)
    {
        var key = parts[0].Trim();
        var value = parts[1].Trim();
        Environment.SetEnvironmentVariable(key, value);
    }
}

string connectionString = "Server=" + Environment.GetEnvironmentVariable("Server") + ";Database=" + Environment.GetEnvironmentVariable("Database") + ";User Id=" + Environment.GetEnvironmentVariable("User") + ";Password=" + Environment.GetEnvironmentVariable("Password") + ";Encrypt=True;TrustServerCertificate=True;Connection Timeout=30;";


In [ ]:
#!connect mssql --kernel-name Pampero @csharp:connectionString

1)	Escriba una consulta que devuelva los pedidos realizados en el último día de actividad que se encuentre en la tabla Pedidos


In [ ]:
SELECT IDPedido, IDCliente, IDEmpleado, FechaPedido
FROM Pedidos AS P
WHERE P.FechaPedido >= (SELECT MAX(DATETRUNC(DAY, PI.FechaPedido)) FROM Pedidos AS PI);

2)	Escribir una consulta que devuelva todos los pedidos realizados por el cliente/s que realizó el mayor número de pedidos.


In [ ]:
SELECT IDCliente, IDPedido, FechaPedido, IDEmpleado
FROM Pedidos
WHERE IDCliente IN
(SELECT TOP (1) WITH TIES P.IDCliente
FROM Pedidos AS P
GROUP BY P.IDCliente
ORDER BY COUNT(*) DESC);

3)	Escriba una consulta que devuelva los empleados que no registraron pedidos luego del 1 de mayo de 2021


In [ ]:
SELECT IDEmpleado, Nombre, Apellido
FROM Empleados
WHERE IDEmpleado NOT IN
(SELECT P.IDEmpleado
FROM Pedidos AS P
WHERE P.FechaPedido >= '20210501');

4)	Escriba una consulta que devuelva los países donde hay clientes, pero no hay empleados.

In [ ]:
SELECT DISTINCT C.Pais
FROM Clientes AS C
WHERE C.Pais NOT IN
(SELECT E.Pais FROM Empleados AS E);

5)	Escriba una consulta que devuelva para cada cliente todos los pedidos realizados en el último día de actividad de ese cliente.

In [ ]:
SELECT IDCliente, IDPedido, FechaPedido, IDEmpleado
FROM Pedidos AS P1
WHERE DATETRUNC(DAY, FechaPedido) =
(SELECT MAX(DATETRUNC(DAY, PS.FechaPedido))
FROM Pedidos AS PS
WHERE PS.IDCliente = P1.IDCliente)
ORDER BY IDCliente;

6)	Escriba una consulta que devuelva los clientes que realizaron pedidos en 2020, pero no en 2021.

In [ ]:
SELECT IDCliente, NombreEmpresa
FROM Clientes AS C
WHERE EXISTS
(SELECT *
FROM Pedidos AS P1
WHERE P1.IDCliente = C.IDCliente
AND P1.FechaPedido >= '20200101'
AND P1.FechaPedido < '20210101')
AND NOT EXISTS
(SELECT *
FROM Pedidos AS P2
WHERE P2.IDCliente = C.IDCliente
AND P2.FechaPedido >= '20210101'
AND P2.FechaPedido < '20220101');

7)	Escriba una consulta que devuelva los clientes que pidieron el producto 12

In [ ]:
SELECT IDCliente, NombreEmpresa
FROM Clientes AS C
WHERE EXISTS
(SELECT *
FROM Pedidos AS P
WHERE P.IDCliente = C.IDCliente
AND EXISTS
(SELECT *
FROM [Detalles Pedido] AS DP
WHERE DP.IDPedido = P.IDPedido
AND DP.IDProducto = 12));

8)	Escriba una consulta que devuelva para cada cliente y mes el total vendido, y el total acumulado hasta ese momento.

In [ ]:
WITH ClientesMes AS
(SELECT
    P.IDCliente,
    DATETRUNC(MONTH, FechaPedido) AS MesOrden,
    SUM(DP.Cantidad) AS cantidad
FROM Pedidos AS P
    JOIN [Detalles Pedido] AS DP
    ON DP.IDPedido = P.IDPedido
GROUP BY IDCliente, DATETRUNC(MONTH, FechaPedido)
)

SELECT
    IDCliente,
    MesOrden,
    cantidad,
    SUM(cantidad) OVER (PARTITION BY IDCliente ORDER BY MesOrden) AS cantacum
FROM ClientesMes
ORDER BY IDCliente, MesOrden;

In [ ]:
WITH ClientesMes AS
(SELECT
    P.IDCliente,
    DATE_BUCKET(MONTH, 1, FechaPedido) AS MesOrden,
    SUM(DP.Cantidad) AS cantidad
FROM Pedidos AS P
    JOIN [Detalles Pedido] AS DP
    ON DP.IDPedido = P.IDPedido
GROUP BY IDCliente, DATE_BUCKET(MONTH, 1, FechaPedido)
)

SELECT
    IDCliente,
    MesOrden,
    cantidad,
    SUM(cantidad) OVER (PARTITION BY IDCliente ORDER BY MesOrden) AS cantacum
FROM ClientesMes
ORDER BY IDCliente, MesOrden;

In [ ]:
SELECT IDCliente, MesOrden, cantidad,
(SELECT SUM(O2.cantidad)
FROM ClientesOrdenes AS O2
WHERE O2.IDCliente = O1.IDCliente
AND O2.MesOrden <= O1.MesOrden) AS cantacum
FROM ClientesOrdenes AS O1
ORDER BY IDCliente, MesOrden;

9)	
a.	Escribir una consulta que genere cinco copias de cada empleado (ID, nombre, apellido, nro de copia)

In [ ]:
SELECT E.IDEmpleado, E.Nombre, E.Apellido, N.n
FROM Empleados AS E
CROSS JOIN dbo.Nums AS N
WHERE N.n <= 5
ORDER BY IDEmpleado, n;

In [ ]:
SELECT E.IDEmpleado, E.Nombre, E.Apellido, N.value as n
FROM Empleados AS E
CROSS JOIN GENERATE_SERIES(1, 5) AS N /* Pag 67 del apunte */
ORDER BY IDEmpleado, value;

b.	Escribir una consulta que devuelva una fila para cada empleado y día entre 12/06/2021 y 16/06/2021 (ID, día)

In [ ]:
SELECT E.IDEmpleado,
DATEADD(day, D.n - 1, CAST('20210612' AS DATE)) AS dt
FROM Empleados AS E
CROSS JOIN dbo.Nums AS D
WHERE D.n <= DATEDIFF(day, '20210612', '20210616') + 1
ORDER BY IDEmpleado, dt;

In [ ]:
SELECT E.IDEmpleado,
DATEADD(day, D.[value] - 1, CAST('20210612' AS DATE)) AS dt
FROM Empleados AS E
CROSS JOIN GENERATE_SERIES(1, DATEDIFF(day, '20210612', '20210616') + 1)  AS D
ORDER BY IDEmpleado, dt;

10)	Explique qué está mal en la siguiente consulta y escriba una alternativa correcta:
```
SELECT Clientes.IDCliente, Clientes.Nombreempresa, Pedidos.IDPedido, Pedidos.FechaPedido
FROM Clientes AS C
INNER JOIN Pedidos AS P
ON Clientes.IDCliente = Pedidos.IDCliente;
```

In [ ]:
-- Cuidado siempre con los Alias
SELECT C.IDCliente, C.Nombreempresa, P.IDPedido, P.FechaPedido
FROM Clientes AS C
INNER JOIN Pedidos AS P
ON C.IDCliente = P.IDCliente;

11)	Devolver los clientes de Estados Unidos (EE.UU.), y para cada cliente el total de pedidos generados y las cantidades totales pedidas (IDCliente, cant pedidos, cantidad total)

In [ ]:
SELECT C.IDCliente, COUNT(DISTINCT P.IDPedido) AS numordenes, SUM(DP.Cantidad) AS cantidadtotal
FROM Clientes AS C
INNER JOIN Pedidos AS P
ON P.IDCliente = C.IDCliente
INNER JOIN [Detalles Pedido] AS DP
ON DP.IDPedido = P.IDPedido
WHERE C.Pais = 'EE.UU.'
GROUP BY C.IDCliente;

12)	Devolver una lista de clientes (ID, nombreempresa) y sus pedidos (ID, FechaPedido). Incluir a los clientes que no hayan generado pedidos.

In [ ]:
SELECT C.IDCliente, C.NombreEmpresa, P.IDPedido, P.FechaPedido
FROM Clientes AS C
LEFT OUTER JOIN Pedidos AS P
ON P.IDCliente = C.IDCliente;

13)	Devolver una lista de los clientes (ID, nombreempres) que no hayan generado pedidos.

In [ ]:
SELECT C.IDCliente, C.NombreEmpresa
FROM Clientes AS C
LEFT OUTER JOIN Pedidos AS P
ON P.IDCliente = C.IDCliente
WHERE P.IDPedido IS NULL;

14)	Devolver una lista de clientes (ID, nombreempresa) y sus pedidos (ID, FechaPedido), con aquellos clientes que hayan efectuado pedidos el 12/02/2021.

In [ ]:
SELECT C.IDCliente, C.NombreEmpresa, P.IDPedido, P.FechaPedido
FROM Clientes AS C
JOIN Pedidos AS P
ON P.IDCliente = C.IDCliente
WHERE P.FechaPedido >= '20210212'
AND P.FechaPedido < '20210213';

15)	Devolver una lista de clientes (ID, nombreempresa) y sus pedidos (ID, FechaPedido), con TODOS los clientes. Para aquellos clientes que hayan efectuado pedidos el 12/02/2021, deberán aparecer los datos de sus pedidos.

In [ ]:
SELECT C.IDCliente, C.NombreEmpresa, P.IDPedido, P.FechaPedido
FROM Clientes AS C
LEFT OUTER JOIN Pedidos AS P
ON P.IDCliente = C.IDCliente AND P.FechaPedido >= '20210212' AND P.FechaPedido < '20210213';

16)	Explique por qué la siguiente no es una solución correcta al ejercicio 15
```
SELECT C.IDCliente, C.NombreEmpresa, P.IDPedido, P.FechaPedido
FROM Clientes AS C
LEFT OUTER JOIN Pedidos AS P
ON P.IDCliente = C.IDCliente
WHERE P.FechaPedido = '20210212'
OR P.IDPedido IS NULL;
```

In [ ]:
SELECT C.IDCliente, C.NombreEmpresa, P.IDPedido, P.FechaPedido
FROM Clientes AS C
LEFT OUTER JOIN Pedidos AS P
ON P.IDCliente = C.IDCliente
WHERE P.FechaPedido >= '20210212' AND P.FechaPedido < '20210213'
OR P.IDPedido IS NULL;

/* Corrigiendo el problema con el campo fecha de todos modos la consulta no es correcta ya que
   se está utilizando un LEFT JOIN y la condición de fecha en el WHERE filtra los resultados
   de la tabla Pedidos, lo que provoca que no se muestren los clientes sin pedidos en esa fecha.
   Para solucionar esto, se debe mover la condición de fecha a la cláusula ON del JOIN. */


17)	Devolver una lista de TODOS los clientes y para cada uno, agregar una columna con Sí o No, dependiendo de si el cliente generó un pedido el 12/02/2021

In [ ]:
SELECT DISTINCT C.IDCliente, C.NombreEmpresa,
CASE WHEN P.IDPedido IS NOT NULL THEN 'Si' ELSE 'No' END AS Ordenel20210212
FROM Clientes AS C
LEFT OUTER JOIN Pedidos AS P
ON P.IDCliente = C.IDCliente
AND P.FechaPedido >= '20210212' AND P.FechaPedido < '20210213';